# 2. Aquifer Detection in Greenland and Antarctica Using C-Band Active Systems

### Introduction

The polar ice sheets of Antarctica and Greenland are undergoing significant changes due to global climate change. One key process influencing these ice sheets is the formation of subglacial water storage areas, known as Perennial Firn Aquifers (PFAs). These aquifers store liquid water in the subsurface firn (snow that has persisted through one or more melting seasons). PFAs play a crucial role in regulating the thermal state of the ice and may impact the overall stability of ice sheets through processes like meltwater-induced hydrofracturing. Understanding PFAs is vital because, as global temperatures rise, the increase in meltwater production across polar regions becomes a significant factor in ice shelf disintegration.

PFAs are typically found in areas with both high melt rates and high snowfall, which allows the water to accumulate below the surface. While they were first discovered in mountain glaciers, scientists have also identified PFAs in Greenland and Svalbard. Several techniques have been used to study PFAs, such as ground and airborne radar, and satellite remote sensing data, particularly using Sentinel-1’s C-band radar imagery. In Antarctica, recent research has started to explore the presence of PFAs through field-based studies and Ground Penetrating Radar (GPR), uncovering aquifers in areas like the Wilkins Ice Shelf.

This project will use C-band active systems to detect aquifers in both Greenland and Antarctica. 
Sentinel-1 (S1) synthetic aperture radar (SAR) has the ability to detect PFA: the detection mechanism relies on the strong absorption of the S1 radar signal by liquid water, potentially at the aquifer water table (when close to the surface; e.g., less than 10 m), but more likely within the firn layer above it. The limited penetration depth at C-band does not always allow sensing the top of the water table itself, which on average, in Greenland, is located 22 m below the surface. However, the time-delayed increase in radar backscatter caused by the delayed refreezing of water in the firn at aquifer locations yields a unique signature in the S1 backscatter data. The long-term continuity in C-band SAR observations with the S1 constellation missions warrants the monitoring of firn aquifers at the decadal time scale.

The S1 detected PFAs will be compared with Operation IceBridge (OIB) data and regional climate models to validate the results. 

We will follow a series of steps to detect and map these aquifers. While the provided code focuses on the Antarctic Peninsula, your task will be to apply the same approach to Greenland using the dataset provided (collected with the same instruments as presented below). Finally, you will be required to validate your results and discuss your findings.



In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import glob


In [ ]:
# In this block, the AP mask is imported with the same resolution/grid as the Sentinel stacks (approximately 4 km),
# and both the mask and the Lat and Lon matrices are reconstructed, which are useful for georeferencing the stacks as well
# (in the 3031 reference system).

import numpy as np
import rasterio
from pyproj import Transformer
import matplotlib.pyplot as plt

# Open the GeoTIFF using rasterio
with rasterio.open('Data/Sentinel-1/maskAP.tif') as dataset:
    # Read the mask data
    maskAP = dataset.read(1)  # Reads the first (and only) layer of the GeoTIFF file
    
    # Get the transformation values to convert from pixel coordinates to geographic coordinates
    transform_affine = dataset.transform
    
    # Extract the x and y coordinate values (in the units of the original projection, e.g., meters in EPSG:3031)
    cols, rows = np.meshgrid(np.arange(maskAP.shape[1]), np.arange(maskAP.shape[0]))
    xs, ys = rasterio.transform.xy(transform_affine, rows, cols)

    # Flatten the coordinate arrays (from 2D to 1D)
    xs_flat = np.array(xs).flatten()
    ys_flat = np.array(ys).flatten()

# Convert the x and y coordinates into geographic coordinates (latitude and longitude)
# We use pyproj to transform the coordinates from EPSG:3031 (Antarctic Polar Stereographic) to EPSG:4326 (lat/lon)

    transformer = Transformer.from_crs("EPSG:3031", "EPSG:4326", always_xy=True)
    lon_flat, lat_flat = transformer.transform(xs_flat, ys_flat)

    # Reshape the latitude and longitude arrays (from 1D to 2D)
    lon = np.array(lon_flat).reshape(maskAP.shape)
    lat = np.array(lat_flat).reshape(maskAP.shape)

# Plot the mask as a function of latitude and longitude coordinates
plt.figure(figsize=(10, 8))
plt.pcolormesh(lon, lat, maskAP, cmap='gray', shading='auto')
plt.colorbar(label='Mask Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Antarctic Peninsula Mask (maskAP)')
plt.show()


### Satellite data description

C-band (5.4 GHz) radar backscatter measurements from Sentinel-1A and Sentinel-1B satellites in extra-wide swath mode are provided for this project. These measurements, collected in dual polarization (horizontal-horizontal and horizontal-vertical), feature a swath width of 410 km and a spatial resolution of 20 by 40 m.

Using the Google Earth Engine Python API, standard preprocessing steps have been performed, including noise removal, radiometric calibration, and terrain correction. The data were aggregated and projected onto a 1 km global grid to enhance the signal-to-noise ratio, particularly for cross-polarized measurements.

Each satellite (S1A and S1B) has a 12-day repeat cycle, with adjustments made for varying incidence and azimuth angles. Datasets were processed separately. This workflow has facilitated reliable backscatter measurements for analyzing firn aquifers.

Now we start by loading the necessary data files, which include georeferenced `.tif` images and associated `.csv` files containing relevant metadata or additional data points. Here's a breakdown of what happens:

1. **Data Collection**: The program searches for all `.tif` and `.csv` files in a predefined folder (`data/`). Using the `glob` function, it locates the file paths for the TIFF images (which contain remote sensing data) and CSV files (likely containing complementary data, such as geographic coordinates or validation information).
   
2. **TIFF Loading**: Each `.tif` file is then opened using `rasterio`, a library that preserves important geospatial information such as Coordinate Reference System (CRS) and the affine transformation needed to map the images to real-world coordinates. The images are read into a list as 3D NumPy arrays, representing multiple bands (layers) of the data. The geospatial metadata for each file is also stored separately.

3. **Data Stacking**: Once the `.tif` files are loaded, the individual layers (bands) from each stack are separated and merged into a single comprehensive stack. This stack will form the basis for further analysis. Each 2D image (a single band) is treated as one layer in this new 3D array.

The final output is a single 3D array containing all the layers (bands) from all `.tif` files, which is ready to be processed for aquifer detection.

In [ ]:
# List of data files (.tif and .csv)
# Define the path to the folder
data_dir = 'Data/Sentinel-1/data/'

# Create two separate lists for .tif and .csv files
tif_files = []
csv_files = []

# Use glob to find .tif and .csv files
tif_files = glob.glob(os.path.join(data_dir, '*.tif'))
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))

# Print the number of files and their names
print(f"Found {len(tif_files)} TIFF files:")
for tif in tif_files:
    print(tif)

print(f"\nFound {len(csv_files)} CSV files:")
for csv in csv_files:
    print(csv)

In [ ]:
# Store all images from the various tif files in a single stack: 
# STEP 1: load the tif files

# List to hold image stacks (each .tif file contains multiple layers)
image_data = []

# List to hold georeferencing information
geo_data = []

# Loop through each .tif file
for tif_file in tif_files:
    # Open the .tif file using rasterio, which preserves georeferencing
    with rasterio.open(tif_file) as src:
        # Read all the layers (bands) in the stack as a NumPy array
        img_stack = src.read()  # This returns a 3D array (bands, rows, cols)
        
        # Store the array of the image stack (do not try to convert to a unified array yet)
        image_data.append(img_stack)
        
        # Store the georeferencing info (such as CRS and affine transform)
        geo_data.append({
            "crs": src.crs,
            "transform": src.transform,
            "bounds": src.bounds
        })

# Output the number of .tif files loaded and their shapes
for idx, img in enumerate(image_data):
    print(f"File {idx+1}: {img.shape} (bands, rows, cols)")

print(f"Loaded {len(image_data)} .tif files.")


In [ ]:
# Step 2: merge into a single stack

# Assuming image_data is a list where each element is a NumPy array of shape (bands, 313, 322)

# Initialize an empty list to hold all the individual images
all_images = []

# Loop through each image stack in image_data
for img_stack in image_data:
    # For each stack, split the images (along the first dimension)
    for i in range(img_stack.shape[0]):
        # Append each individual image (313x322) to the all_images list
        all_images.append(img_stack[i])

# Convert the list of images to a single NumPy array (stacked along the first dimension)
all_images_stack = np.array(all_images)

# Print the new dimensions of the stack
print(f"New stack dimensions: {all_images_stack.shape} (total images, rows, cols)")

In [ ]:
# The dates contained in the csv files are imported and combined into a single date vector. The dates (initially in chronological order by revolution number)
# are reordered chronologically, and using the same criteria, the images are reordered by extracting the indices that correspond to the ordered dates.

# Initialize an empty list to hold individual DataFrames and dates
dataframes = []
date_list = []

# Loop through each CSV file and read it into a DataFrame
for csv_file in csv_files:
    df = pd.read_csv(csv_file)  # Read the CSV file
    dataframes.append(df)  # Append the DataFrame to the list
    
    # Extract the date column and convert to a list
    if 'date' in df.columns:
        date_list.extend(df['date'].tolist())  # Add dates to the date list

# Convert date_list to a pandas Series for sorting
date_series = pd.Series(date_list)

# Sort the dates and get the sorted indices
sorted_indices = date_series.argsort()

# Sort the date list
sorted_dates = date_series.sort_values().reset_index(drop=True)

# Assuming all_images_stack is a NumPy array with shape (3780, 313, 322)
# Reorder the image stack based on the sorted indices
sorted_image_stack = all_images_stack[sorted_indices]

# Print the sorted dates and the new stack dimensions
print("Sorted Dates:")
print(sorted_dates)

print(f"New stack dimensions: {sorted_image_stack.shape} (total images, rows, cols)")

## Backscatter analysis 

In this section, we perform a series of tasks to analyze the backscatter time series for selected pixels on the Antarctic Peninsula:

1. **Pixel Selection**: We define the geographic coordinates (latitude and longitude) for three specific pixels of interest. The goal is to track and analyze the backscatter values at these points over time.

2. **Finding Nearest Pixels**: Using a function that computes the Euclidean distance between the target coordinates and all points in the latitude/longitude grid, the program identifies the nearest corresponding pixels in the dataset.

3. **Extract Time Series**: After locating the pixels, we extract the backscatter time series from the image stack for each of the three selected pixels. These values are stored and indexed by date.

4. **Smoothing the Data**: To reduce noise and provide clearer trends, a 3-week moving average is applied to the time series data for each pixel. This helps in visualizing broader patterns in the backscatter changes over time.

5. **Mapping the Pixels**: Using the Basemap library, we plot the locations of the selected pixels on a map of the Antarctic Peninsula. This step provides a visual reference for where the pixels are located geographically.

6. **Plotting the Time Series**: Finally, the smoothed time series for each pixel is plotted on a graph. This shows the variations in backscatter values over time, giving insights into the dynamic changes at these points.

This block provides both spatial and temporal analysis, linking geographic locations with backscatter time series, essential for understanding aquifer dynamics in the region.

In [ ]:
!pip install basemap

In [ ]:
from mpl_toolkits.basemap import Basemap
from datetime import datetime
import pandas as pd

# Input data: Coordinates of the three selected pixels
pixel1_coords = (-70, -65)  # Latitude, Longitude of pixel1
pixel2_coords = (-70.90, -72.80)  # Latitude, Longitude of pixel2
pixel3_coords = (-70.90, -71.70)  # Latitude, Longitude of pixel3

# Function to find the nearest pixel in the latitude/longitude grid
def find_closest_pixel(lat_grid, lon_grid, target_lat, target_lon):
    """
    Find the closest pixel in the latitude/longitude grid to the target coordinates.
    Calculates the Euclidean distance between the target point and all grid points,
    then returns the index of the nearest pixel.
    """
    # Compute the distance between each grid point and the target point
    dist = np.sqrt((lat_grid - target_lat)**2 + (lon_grid - target_lon)**2)
    # Get the index of the pixel with the smallest distance
    idx = np.unravel_index(np.argmin(dist), lat_grid.shape)
    return idx

# Find the nearest pixel for pixel1, pixel2, and pixel3
pixel1_idx = find_closest_pixel(lat, lon, *pixel1_coords)
pixel2_idx = find_closest_pixel(lat, lon, *pixel2_coords)
pixel3_idx = find_closest_pixel(lat, lon, *pixel3_coords)

# Extract the time series for the selected pixels from the image stack
time_series_pixel1 = sorted_image_stack[:, pixel1_idx[0], pixel1_idx[1]]
time_series_pixel2 = sorted_image_stack[:, pixel2_idx[0], pixel2_idx[1]]
time_series_pixel3 = sorted_image_stack[:, pixel3_idx[0], pixel3_idx[1]]

# Convert sorted_dates to datetime format if it's not already
if isinstance(sorted_dates[0], str):
    sorted_dates_ts = [datetime.strptime(date, '%Y-%m-%d') for date in sorted_dates]

# Create a pandas DataFrame to handle the time series data and apply the moving average
df = pd.DataFrame({
    'Date': sorted_dates_ts,
    'Pixel1': time_series_pixel1,
    'Pixel2': time_series_pixel2,
    'Pixel3': time_series_pixel3
})
df.set_index('Date', inplace=True)

# Apply a moving average of 3 weeks (assuming dates are weekly or spaced similarly)
df_smoothed = df.rolling(window=3, min_periods=1).mean()

# Plot the positions of the three pixels on the map of the Antarctic Peninsula
plt.figure(figsize=(8, 6))

# Use Plate Carrée projection (cylindrical) for clearer, less distorted projection
m = Basemap(projection='cyl', llcrnrlat=-78, urcrnrlat=-58, llcrnrlon=-80, urcrnrlon=-50, resolution='i')

# Draw coastlines, parallels, and meridians
m.drawcoastlines()
m.drawparallels(np.arange(-90, -60, 5), labels=[1, 0, 0, 0])
m.drawmeridians(np.arange(-180, 180, 10), labels=[0, 0, 0, 1])

# Convert lat/lon of the three pixels to map coordinates
x1, y1 = m(pixel1_coords[1], pixel1_coords[0])  # Pixel1 lon, lat
x2, y2 = m(pixel2_coords[1], pixel2_coords[0])  # Pixel2 lon, lat
x3, y3 = m(pixel3_coords[1], pixel3_coords[0])  # Pixel3 lon, lat

# Plot the locations of pixel1, pixel2, and pixel3
m.scatter(x1, y1, color='red', label=f'Pixel1: {pixel1_coords}', s=100, marker='o')
m.scatter(x2, y2, color='blue', label=f'Pixel2: {pixel2_coords}', s=100, marker='o')
m.scatter(x3, y3, color='green', label=f'Pixel3: {pixel3_coords}', s=100, marker='o')

# Add legend to the map
plt.legend(loc='lower left')

# Add a title to the map
plt.title('Locations of Selected Pixels on Antarctic Peninsula')

# Show the map plot
plt.show()

# Now, plot the smoothed time series for all three pixels

plt.figure(figsize=(10, 6))

# Plot time series for Pixel 1 with dashed lines between points
plt.plot(df_smoothed.index, df_smoothed['Pixel1'], label=f'Pixel1 {pixel1_coords}', marker='o', color='red', linestyle='--')

# Plot time series for Pixel 2 with dashed lines between points
plt.plot(df_smoothed.index, df_smoothed['Pixel2'], label=f'Pixel2 {pixel2_coords}', marker='o', color='blue', linestyle='--')

# Plot time series for Pixel 3 with dashed lines between points
plt.plot(df_smoothed.index, df_smoothed['Pixel3'], label=f'Pixel3 {pixel3_coords}', marker='o', color='green', linestyle='--')

# Add title, labels, and legend
plt.title('Smoothed Time Series of Backscatter for Selected Pixels')
plt.xlabel('Date')
plt.ylabel('Backscatter Value (dB)')
plt.legend()
plt.grid(True)

# Rotate date labels for better readability
plt.xticks(rotation=45)
plt.tight_layout()

# Show the time series plot
plt.show()

This section divides the full image stack into annual sub-stacks based on the Antarctic year, which runs from July 1st of one year to June 30th of the following year.

1. **Date Conversion**: We begin by converting the `sorted_dates` list into `Timestamps`, which allows for easier manipulation and filtering of dates. These dates represent when each image in the stack was captured.

2. **Antarctic Year Definition**: The Antarctic year is defined from July 1st of one year to June 30th of the next. Based on the full range of dates in the dataset, we determine the start and end years.

3. **Sub-stack Creation**: For each Antarctic year, a date range is created (e.g., July 1st, 2020 to June 30th, 2021). Using this range, a mask is applied to filter the images and their corresponding dates that fall within that period.

4. **Storing Sub-stacks**: For each Antarctic year, a sub-stack of images and dates is stored in a dictionary. Each sub-stack contains only the images and dates corresponding to that year's date range.

5. **Verification**: After the sub-stacks are created, the shapes of each sub-stack are printed to verify that the division is correct and that each sub-stack contains the expected number of images and date entries.

This process is crucial for analyzing the seasonal dynamics in the Antarctic Peninsula, allowing us to study changes in backscatter and aquifer presence across different years.

In [ ]:
# The images are divided by Antarctic years from 01/07/yyyy to 30/06/yyyy+1

# Assuming sorted_dates is a list or a Series of date strings, convert to Timestamps
sorted_dates = pd.to_datetime(sorted_dates)  # Convert to Timestamps if it's a Series or list

# Define the start and end year based on the date range
start_year = sorted_dates.dt.year.min()  # Get the minimum year
end_year = sorted_dates.dt.year.max()  # Get the maximum year

# Initialize a dictionary to hold sub-stacks for each year
annual_sub_stacks = {}

# Generate date ranges from 1/7/20xx to 30/6/20yy
for year in range(start_year, end_year + 1):
    start_date = pd.Timestamp(year=year, month=7, day=1)
    end_date = pd.Timestamp(year=year + 1, month=6, day=30)

    # Create a mask to filter images based on the current date range
    mask = (sorted_dates >= start_date) & (sorted_dates <= end_date)
    
    # Check if the mask covers the entire range
    if mask.any() and (sorted_dates[mask].min() == start_date) and (sorted_dates[mask].max() == end_date):
        # Create sub-stacks for both images and dates
        annual_sub_stacks[f"{start_date.date()} - {end_date.date()}"] = {
            "images": all_images_stack[mask.values],
            "dates": sorted_dates[mask].values  # Store the corresponding dates
        }

# Output the shape of each sub-stack to verify
for period, stack in annual_sub_stacks.items():
    print(f"Sub-stack for period {period} has shape: {stack['images'].shape} with dates: {stack['dates']}")

### Firn Aquifer Detection Process

The detection of firn aquifers relies on a methodology developed by (Brangers et al., 2020), which analyzes C-band radar backscatter signals, specifically focusing on the σ₀(HV) measurements in wet snow zones. Liquid water within the snow and firn layers significantly influences radar signal behavior, leading to lower σ₀ values during melt events. This is because liquid water absorbs and reflects much of the radar signal, reducing the penetration depth and causing strong decreases in σ₀ during summer melt. 

In areas where firn aquifers are typically found, such as the wet snow zone, strong summer melting saturates the firn, resulting in low summer σ₀. As the liquid water content decreases due to refreezing, σ₀(HV) values increase, as the radar signal penetrates deeper into the firn and encounters more volume scattering from the snow and firn. The σ₀(HV) time series at select locations within the wet snow zone is shown in Figure 1.

![Figure 1: Time series](Data/Sentinel-1/Brangers.jpg)

This figure is an example to illustrates the σ₀(HV) time series at select locations (in Greenland) within the wet snow zone (Brangers et al., 2020).The specific locations of the grid cells are indicated correspond to the following coordinates:

- **Northwest Greenland**: 
  - Aquifer: 76.51°N, 61.75°E 
  - Non-aquifer: 76.61°N, 61.28°E 

- **Southeast Greenland**: 
  - Aquifer: 65.30°N, 41.65°E 
  - Non-aquifer: 65.51°N, 41.98°E 

- **South Greenland**: 
  - Aquifer: 61.77°N, 46.38°E 
  - Non-aquifer: 62.03°N, 46.27°E 

These time series allow us to analyze the differences in backscatter signals between aquifer and non-aquifer locations, providing insights into the seasonal dynamics of the firn aquifers.

------

The refreezing of liquid water in aquifer locations is generally delayed due to the insulating effects of autumn snow accumulation and the release of latent heat during refreezing. This results in a marked delay in the increase of σ₀(HV) throughout autumn compared to other areas where refreezing occurs sooner. Importantly, the radar may not directly sense the water table but can detect the slowdown in refreezing in the upper layers, which serves as a proxy for identifying aquifers.

To effectively map firn aquifers, we employ a detection criterion based on the difference in σ₀(HV) between early autumn and late winter. Specifically, the reduction in σ₀(HV) during early autumn must exceed a threshold of 9.4 dB relative to the values observed near the end of winter. This criterion is derived from the observation that σ₀(HV) remains low in aquifer locations (due to persistent liquid water) and is relatively higher in non-aquifer areas (where refreezing occurs earlier). By applying this methodology, we can reliably identify and map firn aquifers across the studied regions.

In [ ]:
# For each annual stack, filter only the images from March of the year yyyy+1 (early autumn) 
# and October of the year yyyy (late winter) and calculate the pixel-wise averages. 
# The averages are saved in dedicated stacks and represented in georeferenced pcolor figures 
# with a color scale saturated in the range [-30; 0] dB. Additionally, the difference 
# between March yyyy+1 and October yyyy is calculated, which is useful for the application 
# of the Brangers et al. method, and is represented in the range [-10; 10] dB, 
# and also saved in a stack.

# Open the GeoTIFF mask file using rasterio
with rasterio.open('Data/Sentinel-1/maskAP.tif') as dataset:
    maskAP = dataset.read(1)  # Read the first layer of the GeoTIFF file

    # Get the affine transformation values to convert from pixel coordinates to geographic coordinates

    transform_affine = dataset.transform

# Extract the values of the x and y coordinates

    cols, rows = np.meshgrid(np.arange(maskAP.shape[1]), np.arange(maskAP.shape[0]))
    xs, ys = rasterio.transform.xy(transform_affine, rows, cols)

# Flatten the coordinate arrays
    xs_flat = np.array(xs).flatten()
    ys_flat = np.array(ys).flatten()

  # Transform the x and y coordinates into latitude and longitude
    transformer = Transformer.from_crs("EPSG:3031", "EPSG:4326", always_xy=True)
    lon_flat, lat_flat = transformer.transform(xs_flat, ys_flat)

# Reshape the latitude and longitude arrays

    lon = np.array(lon_flat).reshape(maskAP.shape)
    lat = np.array(lat_flat).reshape(maskAP.shape)

# Visualization parameters
vmin = -30  # Common lower limit for the first two plots
vmax = 0    # Common upper limit for the first two plots
diff_vmin = -10  # Lower limit for the difference
diff_vmax = 10   # Upper limit for the difference


# Initialize the stacks for the annual averages
mean_october_stack = []
mean_march_stack = []
difference_stack = []
annual_extremes = []

# Iterate through the annual stacks
for period, stack in annual_sub_stacks.items():
    images = stack["images"]
    dates = pd.to_datetime(stack["dates"])

    # Initialize the lists to hold the October and March images
    october_images = []
    march_images = []

    # Filter the images based on the months
    for date, image in zip(dates, images):
        if date.month == 10:  # Octoeber
            october_images.append(image)
        elif date.month == 3:  # March
            march_images.append(image)

   # Convert the lists into NumPy arrays
    october_images = np.array(october_images)
    march_images = np.array(march_images)

  # Check if there are images for both months
    if len(october_images) > 0 and len(march_images) > 0:
       # Calculate the average for October and March, ignoring NaNs
        mean_october = np.nanmean(october_images, axis=0)
        mean_march = np.nanmean(march_images, axis=0)

        # Calculate the difference (March - October)
        difference = mean_march - mean_october

        # Save the annual averages and extremes
        mean_october_stack.append(mean_october)
        mean_march_stack.append(mean_march)
        difference_stack.append(difference)
        annual_extremes.append((period, mean_october.min(), mean_october.max(), mean_march.min(), mean_march.max()))

        # Plotting as a contour plot
        plt.figure(figsize=(18, 6))

        # Plot of the average - October
        plt.subplot(1, 3, 1)
        plt.pcolormesh(lon, lat, np.ma.masked_array(mean_october, mask=np.isnan(maskAP)), cmap='viridis', vmin=vmin, vmax=vmax, shading='auto')
        plt.title('Mean October Backscattering (dB)')
        plt.colorbar(label='Backscattering (dB)')
        plt.axis('off')

  # Plot of the average - March
        plt.subplot(1, 3, 2)
        plt.pcolormesh(lon, lat, np.ma.masked_array(mean_march, mask=np.isnan(maskAP)), cmap='viridis', vmin=vmin, vmax=vmax, shading='auto')
        plt.title('Mean March Backscattering (dB)')
        plt.colorbar(label='Backscattering (dB)')
        plt.axis('off')

        # Plot of the difference (March - October)
        plt.subplot(1, 3, 3)
        diff_masked = np.ma.masked_array(difference, mask=np.isnan(maskAP))
        plt.pcolormesh(lon, lat, diff_masked, cmap='coolwarm', vmin=diff_vmin, vmax=diff_vmax, shading='auto')
        plt.title('Difference (March - October)')
        plt.colorbar(label='Difference (dB)')
        plt.axis('off')

        plt.suptitle(f'Sub-stack Period: {period}')
        plt.show()
    else:
        print(f"No images for October or March in the period {period}.")

# Convert the stacks to NumPy arrays if necessary
mean_october_stack = np.array(mean_october_stack)
mean_march_stack = np.array(mean_march_stack)
difference_stack = np.array(difference_stack)

# Print the annual extremes
for period, min_oct, max_oct, min_mar, max_mar in annual_extremes:
    print(f"Year: {period}, Mean October Range: ({min_oct}, {max_oct}), Mean March Range: ({min_mar}, {max_mar})")

In [ ]:
# Parameters
treshold_brangers = -9.4  # Threshold for aquifer pixels (this can be edited)


# Initialize the cumulative array to sum the aquifer pixels across all years
# This array will have the same dimensions as the images (lat, lon)

cumulative_aquifers = np.zeros_like(maskAP)

# Iterate through the images of difference_stack and annual_extremes
for i, (difference, year_info) in enumerate(zip(difference_stack, annual_extremes)):
    period = year_info[0]  # Estrai l'anno di riferimento
    
    # Apply the maskAP to the difference images: where maskAP is NaN, difference also becomes NaN
    masked_difference = np.where(np.isnan(maskAP), np.nan, difference)
    
   # Create a binary mask for aquifer pixels: 1 if difference < threshold_brangers, 0 otherwise
    aquifers_mask = np.where(masked_difference < treshold_brangers, 1, 0)
    
# Apply the maskAP again to the binary mask to keep only the valid pixels
    aquifers_mask = np.where(np.isnan(maskAP), np.nan, aquifers_mask)
    
   # Add to the cumulative sum
    cumulative_aquifers += np.nan_to_num(aquifers_mask)  # Use nan_to_num to treat NaNs as 0
    
   # Plot the mask applied to the georeferenced map with lon and lat
    plt.figure(figsize=(10, 8))
    
    # Plot the mask (maskAP) in the background
    plt.pcolormesh(lon, lat, np.ma.masked_array(maskAP, mask=np.isnan(maskAP)), cmap='Greys', shading='auto')
    
# Overlay the aquifer pixels (1 where the threshold is exceeded, 0 where it is not)
    plt.pcolormesh(lon, lat, aquifers_mask, cmap='Blues', shading='auto', alpha=0.6)
    
# Title with the reference year

    plt.title(f"Aquifer pixels during the year: {period}")
    plt.colorbar(label="Aquifer Pixels (1 = aquifer, 0 = not aquifer)")
    plt.axis('off')  # Remove the axes

    
    plt.show()


In [ ]:
# After iterating through all the years, plot the cumulative sum of aquifer pixels
plt.figure(figsize=(10, 8))

# Apply the maskAP to the cumulative sum: NaN pixels in maskAP remain NaN
acquiferi_Brangers = np.where(np.isnan(maskAP), np.nan, cumulative_aquifers)

# Plot the cumulative sum of aquifer pixels
plt.pcolormesh(lon, lat, acquiferi_Brangers, cmap='coolwarm', shading='auto')

# Add a colorbar indicating the number of years a pixel has been aquifer
plt.colorbar(label='Number of years with aquifer pixels')

plt.title('Total years in which the pixels have been aquifer (threshold: 9.4 dB)')
plt.axis('off')  # Remove axes

plt.show()

# Aquifer Pixel Analysis and Visualization

This block of code analyzes and visualizes the presence of aquifer pixels across multiple thresholds using Sentinel-1 radar backscatter data. It iterates through a list of predefined thresholds (`soglie_brangers`) to identify aquifer pixels in the cumulative images, summing the occurrences of aquifer presence for each threshold. 

The first loop specifically calculates the cumulative sum of aquifer pixels for the threshold of -6 dB to determine the global maximum for the colorbar. For each threshold, a map is generated showing the cumulative years a pixel has been classified as an aquifer, with a corresponding colorbar to indicate the frequency of aquifer presence over the study period.


In [ ]:
# List of thresholds to iterate through
brangers_thresholds = [6, 7, 8, 9, 10, 11, 12]

# Initialize a variable to store the global maximum for the colorbar
global_max_acquifers = 0

# First loop: calculate the image with threshold 6 to obtain the limits of the colorbar
for threshold_i in brangers_thresholds:
    # Initialize the cumulative array to sum the aquifer pixels across all years for the current threshold
    cumulative_aquifers = np.zeros_like(maskAP)

    # Iterate through the images of difference_stack and annual_extremes
    for i, (difference, year_info) in enumerate(zip(difference_stack, annual_extremes)):
        period = year_info[0]  # Extract the reference year
        
     # Apply the maskAP to the difference images: where maskAP is NaN, difference also becomes NaN
        masked_difference = np.where(np.isnan(maskAP), np.nan, difference)
        
        # Create a binary mask for aquifer pixels: 1 if difference < threshold, 0 otherwise
        aquifers_mask = np.where(masked_difference < -threshold_i, 1, 0)  # Note: difference < -threshold, since the threshold is negative
        
       # Apply the maskAP again to the binary mask to keep only the valid pixels
        aquifers_mask = np.where(np.isnan(maskAP), np.nan, aquifers_mask)
        
        # Add to the cumulative sum only for valid pixels
        cumulative_aquifers += np.nan_to_num(aquifers_mask)  # Use nan_to_num to treat NaNs as 0

    # In the first loop (threshold 6), find the maximum of the cumulative aquifer pixels and store it
    if threshold_i == 6:
        global_max_acquifers = np.nanmax(cumulative_aquifers)

    # Plotting for each threshold
    plt.figure(figsize=(10, 8))

    # Apply the maskAP to the cumulative sum: NaN pixels in maskAP remain NaN
    cumulative_aquifers_masked = np.where(np.isnan(maskAP), np.nan, cumulative_aquifers)

   # Plot the cumulative sum of aquifer pixels
    plot = plt.pcolormesh(lon, lat, cumulative_aquifers_masked, cmap='Purples', shading='auto', vmin=0, vmax=global_max_acquifers)
    
    # Add a colorbar indicating the number of years a pixel has been aquifer
    plt.colorbar(plot, label='Number of years with aquifer pixels')
    
 # Title that includes the value of the threshold used

    plt.title(f'Total years in which the pixels have been aquifer (Threshold = -{threshold_i} dB)')
    plt.axis('off')  # Remove axes
    
    plt.show()


Now we analyze the number of aquifer pixels for each year in the `difference_stack` dataset across a range of thresholds (from 6 to 14 dB in increments of 0.1 dB). For each year, it creates a binary mask indicating the presence of aquifer pixels based on the condition that the masked difference is less than the negative threshold.

The code initializes a list to store the count of pixels exceeding each threshold, iterates through the specified threshold range, and applies the necessary masks to isolate valid pixels. Finally, it visualizes the relationship between the threshold values and the number of positive aquifer pixels for each year, providing insights into how varying thresholds affect pixel classification.


In [ ]:
# Threshold range from 6 to 14 dB with a step of 0.1

brangers_thresholds = np.arange(6, 14.1, 0.1)

# # Iterate through all the years in the difference_stack
for i, difference_year in enumerate(difference_stack):
    # Check the length of annual_extremes[i] and assign accordingly

    if isinstance(annual_extremes[i], (list, tuple)) and len(annual_extremes[i]) == 2:
        start_year, end_year = annual_extremes[i]
        year_label = f"{start_year} - {end_year}"
    else:
        year_label = f"{annual_extremes[i]}"  # If there is only a single year

        
    # Initialize a list to store the number of pixels that exceed the threshold
    pixel_count_above_threshold = []
    
    # Iterate through the thresholds
    for threshold_i in brangers_thresholds:
       # Apply the maskAP: where maskAP is NaN, difference also becomes NaN
        masked_difference = np.where(np.isnan(maskAP), np.nan, difference_year)
        
        # Create a binary mask for aquifer pixels: 1 if difference < threshold, 0 otherwise
        aquifers_mask = np.where(masked_difference < -threshold_i, 1, 0)  # Note: difference < -threshold

        
        # Apply the maskAP to the binary mask to keep only the valid pixels
        aquifers_mask = np.where(np.isnan(maskAP), np.nan, aquifers_mask)
        
        # Count the aquifer pixels (value 1) ignoring NaNs
        pixel_count = np.nansum(aquifers_mask)
        
    # Add the count to the list
        pixel_count_above_threshold.append(pixel_count)
    
   # Plotting: Number of pixels as a function of the threshold for the current year
    plt.figure(figsize=(10, 6))
    plt.plot(brangers_thresholds, pixel_count_above_threshold, marker='o', linestyle='-', color='b')
    plt.title(f'Number of positive pixels as a function of the threshold (Year {year_label})')
    plt.xlabel('Brangers Threshold (dB)')
    plt.ylabel('Number of positive pixels')
    plt.grid(True)
    plt.show()

## VALIDATION using OIB data

The Operation IceBridge (OIB) mission, conducted by NASA, aims to monitor changes in polar ice and glaciers by bridging the gap between the ICESat missions. Utilizing a combination of airborne instruments, IceBridge provides critical data on ice thickness, surface elevation, and other geophysical parameters across the Arctic and Antarctic regions. The mission has contributed to a better understanding of the dynamics of ice sheets, including seasonal changes, and has been instrumental in validating satellite-based observations. Data from IceBridge, particularly in relation to firn aquifers, serves as a valuable reference to assess the accuracy of remote sensing techniques employed in our analysis. The findings from Miege et al. 2016 highlight the significance of these datasets for improving our understanding of the cryosphere and the impacts of climate change.
These data, as provided by Miege et al.2020 (https://www.usap-dc.org/view/dataset/601390), are used for validating the presence of aquifers on the Wilkins ice Shelf.

In [ ]:
# Import OIB points 

# Path to the CSV file
file_path = 'Data/Sentinel-1/OIB_Wilkins.csv'

# Select only the columns for LATITUDE, LONGITUDE, and DEPTH
columns_to_import = ['LATITUDE', 'LONGITUDE', 'DEPTH (m)']

# Import the data
oib_data_subset = pd.read_csv(file_path, usecols=columns_to_import)

# Show the first few rows
print(oib_data_subset.head())

### Visualization of Depth Data from Operation IceBridge

This code block visualizes depth data from the Wilkins Ice Shelf, collected during the Operation IceBridge mission. 


In [ ]:
# Load the data
# file_path = 'OIB_Wilkins.csv'
data = pd.read_csv(file_path)

# Extract latitude, longitude, and depth
lat_OIB = data['LATITUDE']
lon_OIB = data['LONGITUDE']
depth = data['DEPTH (m)']

# Remove NaN values from lat_OIB, lon_OIB, and depth
valid_data = data.dropna(subset=['LATITUDE', 'LONGITUDE', 'DEPTH (m)'])
lat_OIB = valid_data['LATITUDE']
lon_OIB = valid_data['LONGITUDE']
depth = valid_data['DEPTH (m)']

# Debugging: Print the number of valid points
print(f'Number of valid data points: {len(lat_OIB)}')

# Create a Basemap for the Antarctic Peninsula (Wilkins Ice Shelf area)
plt.figure(figsize=(10, 8))

# Use Plate Carrée projection (cylindrical) for clearer, less distorted projection
m = Basemap(projection='cyl', llcrnrlat=-72, urcrnrlat=-68, llcrnrlon=-75, urcrnrlon=-70, resolution='i')

# Draw coastlines, countries, and parallels/meridians
m.drawcoastlines()
m.drawparallels(range(-90, -60, 2), labels=[1, 0, 0, 0])
m.drawmeridians(range(-180, 180, 5), labels=[0, 0, 0, 1])

# Convert lat_OIB/lon_OIB to map projection coordinates
x, y = m(lon_OIB.values, lat_OIB.values)

# Debugging: Print the x and y coordinates
print(f'x coordinates: {x}')
print(f'y coordinates: {y}')

# Plot the data points with depth represented as color
# Increase the marker size for better visibility
sc = m.scatter(x, y, c=depth, cmap='viridis', marker='o', edgecolor='k', s=40)

# Add colorbar to represent depth
cb = plt.colorbar(sc)
cb.set_label('Depth (m)')

# Add title
plt.title('Wilkins Ice Shelf Depth Representation - Antarctic Peninsula')

# Show plot
plt.show()


# Depth Interpolation from Operation IceBridge Data

This code processes depth data collected from the Operation IceBridge mission. It utilizes latitude and longitude coordinates to interpolate depth values onto a predefined grid. The resulting depth data is then visualized over a specified geographical area, providing insights into the ice shelf dynamics in the Wilkins region.


In [ ]:
from scipy.interpolate import griddata

data = pd.read_csv(file_path)

# Extract latitude, longitude, and depth
lat_OIB = data['LATITUDE']
lon_OIB = data['LONGITUDE']
depth = data['DEPTH (m)']

# Remove NaN values from lat_OIB, lon_OIB, and depth
valid_data = data.dropna(subset=['LATITUDE', 'LONGITUDE', 'DEPTH (m)'])
lat_OIB = valid_data['LATITUDE'].values
lon_OIB = valid_data['LONGITUDE'].values
depth = valid_data['DEPTH (m)'].values

# Assuming maskAP, lat, and lon are already defined matrices (from previous work)
# Let's assume lat and lon are 2D arrays corresponding to the geographical area covered by maskAP

# Interpolate depth values onto the grid defined by lat and lon
interpolated_depth = griddata((lon_OIB, lat_OIB), depth, (lon, lat), method='linear')

# Mask the interpolated depth using maskAP
masked_depth = np.where(np.isnan(maskAP), np.nan, interpolated_depth)

# Plot the interpolated depth over the maskAP
plt.figure(figsize=(10, 8))

# Use Plate Carrée projection (cylindrical) for clearer, less distorted projection
m = Basemap(projection='cyl', llcrnrlat=-72, urcrnrlat=-68, llcrnrlon=-75, urcrnrlon=-70, resolution='i')

# Draw coastlines and parallels/meridians
m.drawcoastlines()
m.drawparallels(range(-90, -60, 2), labels=[1, 0, 0, 0])
m.drawmeridians(range(-180, 180, 5), labels=[0, 0, 0, 1])

# Plot the masked depth
c = m.contourf(lon, lat, masked_depth, cmap='viridis', levels=100)

# Add colorbar
cb = plt.colorbar(c)
cb.set_label('Depth (m)')

# Add title
plt.title('Interpolated OIB Depth over MaskAP - Wilkins Area')

# Show plot
plt.show()


# Visualization of Aquifer Years in the Wilkins Area

Now we compute and visualize the number of years each pixel in the Wilkins area has been classified as an aquifer based on difference data. A predefined threshold is applied to identify aquifer pixels. The results are plotted on a map, with overlay points from the Operation IceBridge (OIB) dataset, providing insights into the spatial distribution of aquifer presence in relation to the OIB data.


In [ ]:
# Assuming maskAP and difference_stack are already defined and contain the relevant data
# Here, difference_stack is assumed to have the years of difference data

# Define the threshold
threshold = 9.4

# Initialize the cumulative array to sum the aquifer pixels across all years for the current threshold
cumulative_aquifers = np.zeros_like(maskAP)

# Iterate through the images of difference_stack and annual_extremes
for i, (difference, year_info) in enumerate(zip(difference_stack, annual_extremes)):
        
        # Apply the maskAP to the difference images: where maskAP is NaN, difference also becomes NaN
        masked_difference = np.where(np.isnan(maskAP), np.nan, difference)
        
        # Create a binary mask for aquifer pixels: 1 if difference < threshold, 0 otherwise
        aquifers_mask = np.where(masked_difference < -threshold, 1, 0)  # Note: difference < -threshold, since the threshold is negative
        
        # Apply the maskAP again to the binary mask to keep only the valid pixels
        aquifers_mask = np.where(np.isnan(maskAP), np.nan, aquifers_mask)
        
       # Add to the cumulative sum only for valid pixels
        cumulative_aquifers += np.nan_to_num(aquifers_mask)  # Use nan_to_num to treat NaNs as 0


# Mask the years_aquifer using maskAP
years_aquifer_masked = np.where(np.isnan(maskAP), np.nan, cumulative_aquifers)

# Create the plot
plt.figure(figsize=(10, 8))

# Use Plate Carrée projection (cylindrical) for clearer, less distorted projection
m = Basemap(projection='cyl', llcrnrlat=-72, urcrnrlat=-68, llcrnrlon=-75, urcrnrlon=-70, resolution='i')

# Draw coastlines and parallels/meridians
m.drawcoastlines()
m.drawparallels(range(-90, -60, 2), labels=[1, 0, 0, 0])
m.drawmeridians(range(-180, 180, 5), labels=[0, 0, 0, 1])

# Plot the years_aquifer_masked
c = m.contourf(lon, lat, years_aquifer_masked, cmap='BuPu', levels=np.arange(0, cumulative_aquifers.max()+1), alpha=0.7)

# Overlay the OIB points
x_OIB, y_OIB = m(lon_OIB, lat_OIB)
m.scatter(x_OIB, y_OIB, color='red', marker='o', label='OIB Points', edgecolor='k', zorder=5)

# Add colorbar for years aquifer
cb = plt.colorbar(c)
cb.set_label('Number of Aquifer Years')

# Add title
plt.title('Number of Aquifer Years - Wilkins Area with OIB Points')

# Show legend
plt.legend()

# Show plot
plt.show()


# Climate Data Integration from RACMO2.3

This script incorporates regional climate model data from RACMO2.3 at a 2 × 2 km resolution for the Antarctic region with the aim of offering a visual comparison of the satellite detected PFA and the climate conditions associated with their occurrence. The dataset includes key climate variables such as accumulation and melt data, which are crucial for identifying regions with potential PFA. These conditions, typically associated with moderate to high surface melt and elevated accumulation rates, are downscaled from the original 27 × 27 km resolution of RACMO2.3p2 (Noël et al., 2023). For Greenland, 1 × 1 km resolution data is provided, downscaled from the 5.5 × 5.5 km RACMO2.3p2 model output (Noël et al., 2019).


In [ ]:
## Loading and Georeferencing Climate Data

#This code reads two TIFF files containing melt and accumulation data for the Antarctic Peninsula, 
#converts their pixel coordinates into geographic coordinates (latitude and longitude), 
#and extracts key information such as dimensions and resolution. The transformation from the projected coordinate system EPSG:3031 to 
#geographic coordinates EPSG:4326 is performed using the `pyproj` library. 
#The script outputs the dimensions and resolution of both datasets for further analysis.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
from pyproj import Transformer

# Paths to the TIFF files (update these if needed)
melt_tif_path = 'Data/Sentinel-1/AP_melt.tif'
accumulation_tif_path = 'Data/Sentinel-1/AP_accumulation.tif'

# Create a transformer to convert from EPSG:3031 to EPSG:4326
transformer = Transformer.from_crs("EPSG:3031", "EPSG:4326", always_xy=True)

# Import the AP_melt.tif and AP_accumulation.tif files
with rasterio.open(melt_tif_path) as melt_src, rasterio.open(accumulation_tif_path) as accumulation_src:
    
    # Extract the dimensions (n_image, row, col) for both files
    melt_shape = (melt_src.count, melt_src.height, melt_src.width)  # (bands, rows, cols)
    accumulation_shape = (accumulation_src.count, accumulation_src.height, accumulation_src.width)

    # Extract the georeferencing information (Affine transformation)
    melt_transform = melt_src.transform
    accumulation_transform = accumulation_src.transform
    
    # Extract resolution (deduced from the affine transform)
    melt_resolution = (melt_transform[0], -melt_transform[4])  # (pixel width, pixel height)
    accumulation_resolution = (accumulation_transform[0], -accumulation_transform[4])

    # Create grids for x and y coordinates (centered coordinates for each pixel)
    cols, rows = np.meshgrid(np.arange(melt_src.width), np.arange(melt_src.height))

    # Convert the row/col indices to georeferenced x/y using the transform
    x_AP, y_AP = rasterio.transform.xy(melt_transform, rows, cols)

    # Convert the georeferenced x/y to lat/lon using the pyproj Transformer
    lon_AP, lat_AP = transformer.transform(np.array(x_AP), np.array(y_AP))

    # Read the melt data
    melt_data = melt_src.read()  # Read all bands

    # Read the accumulation data
    accumulation_data = accumulation_src.read()  # Read all bands

# Print the dimensions and resolution
print(f"Melt TIFF dimensions: {melt_shape}")
print(f"Accumulation TIFF dimensions: {accumulation_shape}")
print(f"Melt TIFF resolution (width, height): {melt_resolution}")
print(f"Accumulation TIFF resolution (width, height): {accumulation_resolution}")

In [ ]:
# Reprojection and plotting of melt flux

# Start from the year 2017 and iterate over each year of melt data
start_year = 2017

# Initialize matrices to store reprojected melt data
melt_reprojected = np.zeros((melt_data.shape[0], lat.shape[0], lat.shape[1]))

# Loop over the melt data for each year
for i in range(melt_data.shape[0]):
    # Get the melt data for the current year
    melt_current = melt_data[i, :, :]
    
    # Flatten the lat_AP, lon_AP, and melt data for interpolation
    lat_AP_flat = lat_AP.flatten()
    lon_AP_flat = lon_AP.flatten()
    melt_flat = melt_current.flatten()
    
    # Interpolate melt data onto the new lat/lon grid
    melt_interpolated = griddata(
        (lon_AP_flat, lat_AP_flat),  # Original lon/lat coordinates
        melt_flat,                   # Original melt data values
        (lon, lat),        # New grid of lon/lat to interpolate onto
        method='linear'              # Interpolation method
    )

    # Store the reprojected melt flux data in the matrix
    melt_reprojected[i, :, :] = melt_interpolated
    
    # Plot the reprojected melt data using Basemap
    plt.figure(figsize=(10, 8))

    # Create the Basemap for the Antarctic Peninsula with the given limits
    m = Basemap(projection='cyl', 
                llcrnrlat=-78, urcrnrlat=-58, 
                llcrnrlon=-80, urcrnrlon=-50, 
                resolution='i')

    # Draw coastlines, parallels, and meridians
    m.drawcoastlines()
    m.drawparallels(np.arange(-90, -50, 5), labels=[1, 0, 0, 0])
    m.drawmeridians(np.arange(-180, 180, 10), labels=[0, 0, 0, 1])

    # Plot the interpolated melt data on the Basemap
    x, y = m(lon, lat)
    cs = m.contourf(x, y, melt_interpolated, cmap='coolwarm', levels=np.linspace(0, 800, 100))

    # Add a colorbar with the label for melt in mm w.e./year
    cb = plt.colorbar(cs)
    cb.set_label('mm w.e./year')

    # Add the title with the corresponding year
    plt.title(f'Melt Flux in AP for the year {start_year + i}')

    # Show the plot
    plt.show()

In [ ]:
# Reprojection and plotting of accumulation

# Start from the year 2017 and iterate over each year of accumulation data
start_year = 2017

# Initialize matrices to store reprojected accumulation data
accumulation_reprojected = np.zeros((accumulation_data.shape[0], lat.shape[0], lat.shape[1]))

# Loop over the accumulation data for each year
for i in range(accumulation_data.shape[0]):
    # Get the accumulation data for the current year
    accumulation_current = accumulation_data[i, :, :]
    
    # Flatten the lat_AP, lon_AP, and accumulation data for interpolation
    lat_AP_flat = lat_AP.flatten()
    lon_AP_flat = lon_AP.flatten()
    accumulation_flat = accumulation_current.flatten()
    
    # Interpolate accumulation data onto the new lat/lon grid
    accumulation_interpolated = griddata(
        (lon_AP_flat, lat_AP_flat),  # Original lon/lat coordinates
        accumulation_flat,           # Original accumulation data values
        (lon, lat),                  # New grid of lon/lat to interpolate onto
        method='linear'              # Interpolation method
    )

    # Store the reprojected accumulation data in the matrix
    accumulation_reprojected[i, :, :] = accumulation_interpolated
    
    # Plot the reprojected accumulation data using Basemap
    plt.figure(figsize=(10, 8))

    # Create the Basemap for the Antarctic Peninsula with the given limits
    m = Basemap(projection='cyl', 
                llcrnrlat=-78, urcrnrlat=-58, 
                llcrnrlon=-80, urcrnrlon=-50, 
                resolution='i')

    # Draw coastlines, parallels, and meridians
    m.drawcoastlines()
    m.drawparallels(np.arange(-90, -50, 5), labels=[1, 0, 0, 0])
    m.drawmeridians(np.arange(-180, 180, 10), labels=[0, 0, 0, 1])

    # Plot the interpolated accumulation data on the Basemap
    x, y = m(lon, lat)
    cs = m.contourf(x, y, accumulation_interpolated, cmap='coolwarm', levels=np.linspace(0, 2000, 100))

    # Add a colorbar with the label for accumulation in mm w.e./year
    cb = plt.colorbar(cs)
    cb.set_label('mm w.e./year')

    # Add the title with the corresponding year
    plt.title(f'Accumulation in AP for the year {start_year + i}')

    # Show the plot
    plt.show()

In [ ]:
# Multi-year average values: plot

# Calculate the mean of reprojected melt and accumulation data
mean_melt_reprojected = np.mean(melt_reprojected, axis=0)
mean_accumulation_reprojected = np.mean(accumulation_reprojected, axis=0)

# Create a subplot to visualize mean melt and accumulation
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot for mean melt
melt_map = Basemap(projection='cyl', llcrnrlat=-78, urcrnrlat=-58, llcrnrlon=-80, urcrnrlon=-50, resolution='i', ax=axs[0])
melt_map.drawcoastlines()
melt_map.drawparallels(np.arange(-90, -50, 5), labels=[1, 0, 0, 0])
melt_map.drawmeridians(np.arange(-180, 180, 10), labels=[0, 0, 0, 1])

melt_cs = melt_map.contourf(lon, lat, mean_melt_reprojected, cmap='coolwarm', levels=np.linspace(0, 800, 100))

# Add colorbar for melt
melt_cb = plt.colorbar(melt_cs, ax=axs[0])
melt_cb.set_label('Melt Flux (mm w.e./year)')
axs[0].set_title(f'Melt Flux: Mean over 2017-2021')

# Plot for mean accumulation
accumulation_map = Basemap(projection='cyl', llcrnrlat=-78, urcrnrlat=-58, llcrnrlon=-80, urcrnrlon=-50, resolution='i', ax=axs[1])
accumulation_map.drawcoastlines()
accumulation_map.drawparallels(np.arange(-90, -50, 5), labels=[1, 0, 0, 0])
accumulation_map.drawmeridians(np.arange(-180, 180, 10), labels=[0, 0, 0, 1])

accumulation_cs = accumulation_map.contourf(lon, lat,  mean_accumulation_reprojected, cmap='coolwarm', levels=np.linspace(0, 2000, 100))

# Add colorbar for accumulation
accumulation_cb = plt.colorbar(accumulation_cs, ax=axs[1])
accumulation_cb.set_label('Accumulation (mm w.e./year)')
axs[1].set_title(f'Accumulation: Mean over 2017-2021')

# Set a main title for the subplot
plt.suptitle('Melt and Accumulation Mean Values 2017-2021', fontsize=16)

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Comparison of S1 PFA with pixels of high melt flux and high accumulation

# Assume mean_melt_reprojected, mean_accumulation_reprojected, and acquiferi_Brangers are already defined

# Define thresholds for masking
melt_threshold = 200
accumulation_threshold = 400

# Create a mask where mean melt is greater than the threshold and mean accumulation is greater than the threshold
mask = (mean_melt_reprojected > melt_threshold) & (mean_accumulation_reprojected > accumulation_threshold)

# Apply the mask to acquiferi_Brangers
# Assuming acquiferi_Brangers has the same shape as mean_melt_reprojected and mean_accumulation_reprojected
masked_acquiferi = np.where(mask, acquiferi_Brangers, np.nan)  # Use NaN to hide masked values in the plot

# Create a Basemap for the Antarctic Peninsula
plt.figure(figsize=(10, 8))
m = Basemap(projection='cyl', 
             llcrnrlat=-78, urcrnrlat=-58, 
             llcrnrlon=-80, urcrnrlon=-50, 
             resolution='i')

# Draw coastlines, parallels, and meridians
m.drawcoastlines()
m.drawparallels(np.arange(-90, -50, 5), labels=[1, 0, 0, 0])
m.drawmeridians(np.arange(-180, 180, 10), labels=[0, 0, 0, 1])

# Plot the masked aquifers data
cs = m.contourf(lon, lat, masked_acquiferi, cmap='coolwarm', levels=np.linspace(np.nanmin(masked_acquiferi), np.nanmax(masked_acquiferi), 100))

# Add colorbar with label
cb = plt.colorbar(cs)
cb.set_label('S1 PFA (years)')

# Add title
plt.title('Aquifers According to S1 for High Melt and Accumulation Points')

# Show the plot
plt.show()

# End of the guided topic.

This marks the conclusion of the guided section of this tutorial.
Please analyze the results obtained and find a way to present them to your colleagues during the final presentation. You may use all the tools suggested in this guided part, but feel free to add your comments, analyses, or further ideas for additional investigations.

Once you have commented on the results obtained in Antarctica, you can proceed with aquifer detection in Greenland using the same method outlined here. You will find a folder containing all the necessary data:
a main folder with mask, accumulation, and melt, as well as a subfolder with individual years for Sentinel in .tif and .csv formats. The individual years for S1 are 2019, 2020, 2021, 2022, and 2023, while RACMO (melt and accumulation) is available for 2019, 2020, 2021, and 2022.

Since we are now in a different hemisphere from Antarctica, the definition of 'year' and the implementation of the Brangers thresholds used in our method will need to be adjusted accordingly. As an approximation, everything can be shifted by six months. Please refer to Brangers 2020 for further details.

Additionally, note that the reference system for the Greenland data is EPSG:5938, with a 4 km grid.